# Praca domowa numer 4

## zadanie 0 - wczytanie funkcji

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import tempfile
import os
import sqlite3

In [2]:
Badges    = pd.read_csv('/home/kuba/Documents/Dane/Badges.csv')
Comments  = pd.read_csv('/home/kuba/Documents/Dane/Comments.csv')
PostLinks = pd.read_csv('/home/kuba/Documents/Dane/PostLinks.csv')
Posts     = pd.read_csv('/home/kuba/Documents/Dane/Posts.csv')
Tags      = pd.read_csv('/home/kuba/Documents/Dane/Tags.csv')
Users     = pd.read_csv('/home/kuba/Documents/Dane/Users.csv')
Votes     = pd.read_csv('/home/kuba/Documents/Dane/Votes.csv')

baza = os.path.join(tempfile.mkdtemp(), "baza.db")

if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn) 
PostLinks.to_sql("PostLinks", conn) 
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn) 
Users.to_sql("Users", conn) 
Votes.to_sql("Votes", conn) 


## Zadanie 1 

*Wybierzmy dziesięć najpopularniejszych pytań (jeśli PostTypeId ==1, to jest to pytanie), liczby favorites jakie dostały i informacje o
ich postujących. Chcemy, żeby było uszeregowane od ilości sumy favorites, jakie dostał użytkownik kiedykolwiek.*

-**SQL**

In [3]:
df1_sql = pd.read_sql_query("""
SELECT
Users.DisplayName,
Users.Age,
Users.Location,
SUM(Posts.FavoriteCount) AS FavoriteTotal,
Posts.Title AS MostFavoriteQuestion,
MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
""", conn)

-**Pandas**

In [23]:
# łączę obie tabelki w jedą
df = pd.merge(Users, Posts, left_on='Id', right_on='OwnerUserId')

#Wybieram te kolumny, z którymi będę coś jeszcze robił
df1 = df[['DisplayName', 'Age', 'Location', 'FavoriteCount', 'Title','OwnerUserId', 'PostTypeId']]

# grupuję
df2 = df1.groupby('OwnerUserId')

# wykonuję operacje na grupach
df3 = df2.FavoriteCount.agg([sum, max]).merge(df1,
                                              left_on=['OwnerUserId','max'], 
                                              right_on = ['OwnerUserId','FavoriteCount']).rename(
    columns = {  'sum':'FavoriteTotal',
                 'max':'MostFavoriteQuestionLikes',
                 'Title':'MostFavoriteQuestion'})

# filtruję i sortuję, biorę 10 pierszych i resetuję index
df4 = df3[df3.PostTypeId == 1].sort_values('FavoriteTotal', ascending=False).head(10).reset_index()

# wybieram te kolumny co w zapytaniu sql
df5 = df4[['DisplayName','Age', 'Location','FavoriteTotal','MostFavoriteQuestion',"MostFavoriteQuestionLikes" ]]

# zamieniam Nan na None
df5.Location = df5.Location.where(pd.notnull(df5.Location), None)


In [24]:
#sprawdźmy czy wyszło nam to samo 
df5.equals(df1_sql)

True

# Zadanie 2

*Wybierzmy te pytania, które miały najwięcej pozytywnych odpowiedzi.*

-**SQL** 

In [5]:
df2_sql = pd.read_sql_query("""SELECT
Posts.ID,
Posts.Title,
Posts2.PositiveAnswerCount
FROM Posts
JOIN (
SELECT
Posts.ParentID,
COUNT(*) AS PositiveAnswerCount
FROM Posts
WHERE Posts.PostTypeID=2 AND Posts.Score>0
GROUP BY Posts.ParentID
) AS Posts2
ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10""", conn)

-**Pandas**

In [6]:
# Wybieram z postów pytania, które miały dodatni Score
Posts2 = Posts.loc[(Posts.PostTypeId == 2) & (Posts.Score >0 )]

# Liczę ilość odpowiedzi 
df2 =    Posts2.groupby('ParentId').size().to_frame().rename( columns = {0 : 'PositiveAnswerCount'})

# Merguję z Posts i dzięki temu dla każdego pytania mam ilość pozytywnych odpowiedzi,
# wybieram 10 z największą ilością
df3 =    df2.merge( Posts , left_on = 'ParentId', right_on = 'Id')[[
            'Id','Title', 'PositiveAnswerCount']].reset_index().sort_values([
            'PositiveAnswerCount', 'Id'],ascending = [False,True]).reset_index()[[
            'Id','Title','PositiveAnswerCount']].head(10)

# sprawdzam czy dostaję to samo: 
df3.equals(df2_sql)

True

## Zadanie 3

*Wybierzmy pytania z największą liczbą pozytywnych głosów w danym roku* 

-**SQL**

In [7]:
df3_sql = pd.read_sql_query("""SELECT
Posts.Title,
UpVotesPerYear.Year,
MAX(UpVotesPerYear.Count) AS Count
FROM (
SELECT
PostId,
COUNT(*) AS Count,
STRFTIME('%Y', Votes.CreationDate) AS Year
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, Year
) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year""", conn)

-**Pandas** 

In [8]:
# wybieram te głosy, które były pozytywne
df1 = Votes.loc[Votes.VoteTypeId == 2]

# z daty ekstraktuję rok
df1.CreationDate = df1.CreationDate.str[:4]

# zmieniam nazwę kolumny 
df1 = df1.rename(columns = {'CreationDate':'Year'})

# Liczę upvotes dla danego posta w roku 
UpvotesPerYear = df1.groupby(['PostId', 'Year']).size().to_frame('Count').reset_index()

# merguję wynik z Postami
df2 = pd.merge(UpvotesPerYear, Posts, left_on='PostId', right_on='Id')

# Wybieram z postów pytania
df2 = df2.loc[df2.PostTypeId == 1]

# Liczę maksymalną ilość głosów w roku i merguję z df2, aby dopasować pytanie do liczby głosów
df3 = df2.groupby('Year').Count.agg(max).to_frame('Count').reset_index().merge(df2, on=['Year', 'Count'])

# wybieram tylko 3 kolumny
df4 = df3[['Title','Year', 'Count']]

In [9]:
df4.equals(df3_sql)

True

## Zadanie 4 

*Wybieramy te pytania, na które odpowiedzi, które pojawiają się na semej górze mają o co najmniej 50 puntków mniej niż najbardziej
puntkowana odpowiedź pod tym pytaniem. Szeregujemy malejąco po róźnicy.*

-**SQL** 

In [10]:
df4_sql = pd.read_sql_query("""SELECT
Questions.Id,
Questions.Title,
BestAnswers.MaxScore,
Posts.Score AS AcceptedScore,
BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
SELECT Id, ParentId, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId==2
GROUP BY ParentId
) AS BestAnswers
JOIN (
SELECT * FROM Posts
WHERE PostTypeId==1
) AS Questions
ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC""", conn)

-**Pandas** 

In [11]:
# wybieram z postów odpowedzi, grupuję po Id pytania i wybieram odpowiedź na pytanie z maxScorem
BestAnswers =    Posts.loc[Posts.PostTypeId == 2].groupby(
                'ParentId').Score.agg(max).to_frame('MaxScore').reset_index().merge(
                Posts, left_on=['ParentId', 'MaxScore'], right_on=['ParentId', 'Score'])

# Wybieram 3 kolumny 
BestAnswers = BestAnswers[['Id','ParentId', 'MaxScore']]

# Z postów wybieram pytania
Questions = Posts.loc[Posts.PostTypeId == 1]

# Merguję pytania z odpowiedziami
df1 = pd.merge(Questions, BestAnswers, left_on='Id', right_on = 'ParentId'
              ).rename(columns = {'Score':'QScore'})

# a później z Postami
df2 = pd.merge(df1, Posts, left_on = 'AcceptedAnswerId', right_on='Id').reset_index(
        )[['Id_x','Title_x','Score','MaxScore']]

# liczę różnicę między odpowiedzią mającą maksymalną ilość Scora, a tą z AcceptedScore
df2['Difference'] = df2.MaxScore - df2.Score

# Sortuję po wielkości tej różnicy i porządkuję
df3 = df2.sort_values('Difference', ascending = False).reset_index()[[
       'Id_x','Title_x','MaxScore', 'Score','Difference']].rename(
                                                         columns = {'Score':'AcceptedScore',
                                                                    'Id_x':'Id',
                                                                    'Title_x':'Title'})

# na koniec wybieram te, które mają różnicę międzi maksymalnym a zaakceptowanym Scorem odpowiedzi równą 50
df4 = df3.loc[df3.Difference > 50]

In [12]:
# sprawdzam czy jest takie samo
df4.equals(df4_sql)

True

## Zadanie 5 

*Wybierzmy 10 pytań, których suma głosów dla odpowiedzi była największa.*

-**SQL** 

In [13]:
df5_sql = pd.read_sql_query("""SELECT
Posts.Title,
CmtTotScr.CommentsTotalScore
FROM (
SELECT
PostID,
UserID,
SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10""", conn)

-**Pandas**

In [14]:
# Grupuję po Id posta i Usera, następnie liczę sumę Scora jaką komentarz uzyskał
CmtTotScr = Comments.groupby(['PostId','UserId']).Score.agg(sum).to_frame('CommentsTotalScore').reset_index()

# Merguję z Postami
df1 = pd.merge(CmtTotScr, Posts, left_on=['PostId','UserId'], right_on=['Id','OwnerUserId'])

# Wybieram pytania i sortuję po Score komentarzy
df2 = df1.loc[df1.PostTypeId == 1].sort_values(
    'CommentsTotalScore',ascending = False).reset_index().sort_values(
    ['CommentsTotalScore','index'], ascending=[False,True]).reset_index()[['Title','CommentsTotalScore']].head(10)


In [15]:
# patrzę, czy wyniki są równoważne
df2.equals(df5_sql)

True

## Zadanie 6 

*Wybierzmy tych użytkowników i informacje o nich, którzy mają złotą odznakę, a identyczne odznaki jak oni, ma jedynie od 2 do 10
użytkowników.*

-**SQL** 

In [16]:
df6_sql = pd.read_sql_query("""SELECT
DISTINCT
Users.Id,
Users.DisplayName,
Users.Reputation,
Users.Age,
Users.Location
FROM (
SELECT
Name, UserID
FROM Badges
WHERE Name IN (
SELECT
Name
FROM Badges
WHERE Class=1
GROUP BY Name
HAVING COUNT(*) BETWEEN 2 AND 10
)
AND Class=1
) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id""", conn)

-**Pandas**

In [17]:
# wybierami złotą odznakę i patrzymy ile ich jest
df1 = Badges.loc[Badges.Class == 1].groupby('Name').size().to_frame('Count').reset_index()

# wybieramy te odznaki, które ma od 2 do 10 urzytkowników
df2 = df1.loc[(df1.Count >=2) & (df1.Count <= 10) ][['Name']]

# Wybieramy te odznaki, które są dość rzadkie i są złote
ValuableBadges = Badges.loc[Badges.Name.isin(df2.Name)].loc[Badges.Class == 1][['Name','UserId']]

# Na koniec łączymy Userów, i bierzemy userów, którzy mają te odznaki, patrzymy na dane ich kont.
df3 = pd.merge(ValuableBadges, Users, left_on = 'UserId', right_on='Id')[['Id','DisplayName','Reputation','Age',
                                                                          'Location']].drop_duplicates(
                                                                            ).reset_index(drop=True)

In [18]:
# na koniec patrzymy czy wyniki są równoważne
df3.equals(df6_sql)

True

## Zadanie 7 

*Wybierzmy 10 pytań które dostały najwięcej pozytywnych głosów, stworzonych przed 2016 rokiem, które nie dostały żadnego pozy-
tywnego głosu po 2016 roku.* 

-**SQL** 

In [19]:
df7_sql = pd.read_sql_query("""SELECT
Posts.Title,
VotesByAge2.OldVotes
FROM Posts
JOIN (
SELECT
PostId,
MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
SUM(Total) AS Votes
FROM (
SELECT
PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2017' THEN 'new'
WHEN '2016' THEN 'new'
ELSE 'old'
END VoteDate,
COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, VoteDate
) AS VotesByAge
GROUP BY VotesByAge.PostId
HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10""", conn)

-**Pandas**

In [20]:
# wybieram pozytywne głosy
df1 = Votes.loc[Votes.VoteTypeId == 2]

# z daty biorę rok
df1.CreationDate = df1.CreationDate.str[:4]

# jeśli tok >= 2016 to jest to nowy post, jeśli powstał przed 2016, to jest on stary
df1.CreationDate = ['new' if e in ['2016','2017'] else 'old' for e in df1.CreationDate]
df2 = df1.rename(columns = {'CreationDate':'VoteDate'})

# Grupuję po dacie i Id posta (używam unstacka, bo size() daje nam tylko te obserwacje,
# które występują przynajmniej raz)
VotesByAge = df2.groupby(['PostId','VoteDate']).size().unstack(fill_value=0).stack().to_frame('Total').reset_index()

# liczę ilość nowych głosów dla posta
temp1 = VotesByAge.loc[VotesByAge.VoteDate == 'new'].groupby('PostId').Total.agg(max).to_frame(
                                                                'NewVotes').reset_index()
# teraz ilość starych głosów dla posta
temp2 = VotesByAge.loc[VotesByAge.VoteDate == 'old'].groupby('PostId').Total.agg(max).to_frame(
                                                                'OldVotes').reset_index()

# Merguję te ilości głosów
VotesByAge2 = pd.merge(temp1, temp2, on='PostId')

# teraz wybieram te, które nie mają wcale nowych głosów
VotesByAge2 = VotesByAge2.loc[VotesByAge2.NewVotes == 0]

# merguję wynik z postami
df3 = pd.merge(VotesByAge2, Posts , left_on = 'PostId', right_on = 'Id')

# następnie wybieram te pytania, które mają największą liczbę starych głosów
df4 = df3.loc[df3.PostTypeId == 1][['Title', 'OldVotes']].sort_values('OldVotes', ascending = False).reset_index(
                                    ).sort_values(['OldVotes','index'], ascending = [False, True])[[
                                    'Title', 'OldVotes']].head(10).reset_index(drop=True)


In [21]:
# sprawdzam czy wyniki są równoważne: 
df4.equals(df7_sql)

True